In [1]:
import pandas as pd
import numpy as np

from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score

import joblib

from data_preprocessing import pipeline
from feature_preprocessing_EG import pipeline2

### 데이터 전처리

In [2]:
# 로우 데이터 불러오기
file_path = "./storage/raw_data/병합_청약매물_목록_정보_픽스2.csv"
df = pd.read_csv(file_path)

# 데이터 전처리
preprocessing_pipeline = pipeline(type)
df = preprocessing_pipeline.transform(df)

# 파일 저장
file_version = "250318-유진3"
output_file = f"./storage/train_data/train-{file_version}.csv"
df.to_csv(output_file, index=False, encoding='cp949')

C:\Users\user\AppData\Local\Temp\ipykernel_6616\1366467655.py:3: DtypeWarning: Columns (48) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path)
c:\Users\user\mid-project\src\data_preprocessing.py:146: DtypeWarning: Columns (36,48) have mixed types. Specify dtype option on import or set low_memory=False.
  df_estate_price = pd.read_csv(csv_path, encoding="cp949")
c:\Users\user\mid-project\src\data_preprocessing.py:333: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['최저당첨가점']

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17250 entries, 0 to 17249
Data columns (total 31 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   공급지역코드        17250 non-null  int64  
 1   공급규모          17250 non-null  int64  
 2   투기과열지구        17250 non-null  object 
 3   조정대상지역        17250 non-null  object 
 4   분양가상한제        17250 non-null  object 
 5   정비사업          17250 non-null  object 
 6   공공주택지구        17250 non-null  object 
 7   대규모택지개발지구     17250 non-null  object 
 8   수도권내민영공공주택지구  17250 non-null  object 
 9   공급세대수         17250 non-null  int32  
 10  순위            17250 non-null  object 
 11  거주지역          17250 non-null  object 
 12  접수건수          17250 non-null  int32  
 13  경쟁률           17250 non-null  float64
 14  최저당첨가점        17250 non-null  float64
 15  토픽 1          17250 non-null  float64
 16  토픽 2          17250 non-null  float64
 17  토픽 3          17250 non-null  float64
 18  토픽 4          17250 non-nu

### 모델 학습 및 평가

In [4]:
# 전처리된 데이터 가져오기

# file_version = "250317-01"
# file_path = f"./storage/train_data/t``ain-{file_version}.csv"
# df = pd.read_csv(file_path, encoding='cp949')

df = pd.read_csv("C:/Users/user/mid-project/src/storage/train_data/train-250318-유진3.csv", encoding='cp949')



In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17250 entries, 0 to 17249
Data columns (total 31 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   공급지역코드        17250 non-null  int64  
 1   공급규모          17250 non-null  int64  
 2   투기과열지구        17250 non-null  object 
 3   조정대상지역        17250 non-null  object 
 4   분양가상한제        17250 non-null  object 
 5   정비사업          17250 non-null  object 
 6   공공주택지구        17250 non-null  object 
 7   대규모택지개발지구     17250 non-null  object 
 8   수도권내민영공공주택지구  17250 non-null  object 
 9   공급세대수         17250 non-null  int64  
 10  순위            17250 non-null  object 
 11  거주지역          17250 non-null  object 
 12  접수건수          17250 non-null  int64  
 13  경쟁률           17250 non-null  float64
 14  최저당첨가점        17250 non-null  float64
 15  토픽 1          17250 non-null  float64
 16  토픽 2          17250 non-null  float64
 17  토픽 3          17250 non-null  float64
 18  토픽 4          17250 non-nu

In [6]:
X = df.drop(columns=["최저당첨가점"])
Y = df["최저당첨가점"]

X_train, X_test, y_train, y_test = train_test_split(
    X,  # 타겟(최저당첨가점) 제외
    Y,  # 타겟
    test_size=0.2,
    random_state=42,
    stratify=df["투기과열지구"]  # 분포 맞추기 위해 미달여부로 stratify
)

In [7]:
feature_pipeline = pipeline2()

# 학습 데이터(X_train)에 fit
feature_pipeline.fit(X_train)

# 학습 데이터(X_train)를 변환
X_train_transformed = feature_pipeline.transform(X_train)

# 테스트 데이터(X_test)를 변환
X_test_transformed = feature_pipeline.transform(X_test)

In [8]:
# TODO: 여러 모델 학습 및 하이퍼파라미터 최적화

model = RandomForestRegressor(
    n_estimators=100,
    max_depth=None,
    random_state=42
)

# 모델 학습
model.fit(X_train_transformed, y_train)

# 예측
y_pred = model.predict(X_test_transformed)

# 평가 (RMSE와 R^2 계산)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
r2 = r2_score(y_test, y_pred)

print("RMSE:", rmse) # 낮을수록 좋음
print("R^2 Score:", r2) # 1에 가까울수록 좋음

ValueError: could not convert string to float: '양천구'

### 모델 저장 

In [ ]:
# 모델 저장 및 로드

version = '0.0.1'

# 모델 저장
joblib.dump(model, f"./storage/trained_model/model_{version}.pkl")

['./storage/trained_model/model_0.0.1.pkl']

In [ ]:
# 모델 로드
loaded_model = joblib.load("./storage/trained_model/model_0.0.1.pkl")

# 예측
# X_test = np.array([[1, 2]])
# prediction = loaded_model.predict(X_test)
# print("Prediction:", prediction)